## Random Acts of Pizza

W207 Final Project

Erika Lawrence, Leslie Teo, Jen Jen Chen, Geoff Stirling


_"The universe is hilarious. Like, Venus is 900 degrees. I could tell you it melts lead. But that's not as fun as saying, 'You can cook a pizza on the windowsill in nine seconds.' And next time my fans eat pizza, they're thinking of Venus!"_

_- Neil deGrasse Tyson_ 

----------------------------------------------------------------------

## Section 1: Setting Up & Processing Data

In [16]:
## Import Libraries ##

import json
from pprint import pprint
from pandas import *
from pandas.io.json import json_normalize
from vaderSentiment import vaderSentiment

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import mlxtend
import scipy

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import recall_score

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.metrics import roc_curve, roc_auc_score

# SK-learn libraries for pre/processing data
from sklearn import preprocessing
from sklearn.decomposition import LatentDirichletAllocation as LDA

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# Miscellaneous libraries
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt

In [12]:
## Get Data ##

# Reference for data: https://www.kaggle.com/c/random-acts-of-pizza/data
# Pull in the training and test data
with open('data/train.json', encoding='utf-8') as data_file:
    trainData = json.loads(data_file.read())   

with open('data/test.json', encoding='utf-8') as data_file:
    testData = json.loads(data_file.read())    

# Create a dev data set 
devData = trainData[0:1000]
trainData = trainData[1000:]

# Show how the data looks in its original format
#pprint("data in json format:")
#pprint(trainData[1])

# Create a normalized view
allTData = json_normalize(trainData)
print("\nSize of the normalized Data:", allTData.shape)
print("\nnormalized data columns:", list(allTData))

allDData = json_normalize(devData)


Size of the normalized Data: (3040, 32)

normalized data columns: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_request

In [ ]:
## Create subsets of data for analysis ###

# Create a flat dataset without the subreddits list
flatData = allTData.drop('requester_subreddits_at_request', 1)
# Create a separate dataset with just subreddits, indexed on request id
# We can creata a count vector on the words, run Naive Bayes against it, 
# and add the probabilities to our flat dataset
subredTData = allTData[['request_id','requester_subreddits_at_request']]
subredTData.set_index('request_id', inplace=True)

subredDData= allDData[['request_id','requester_subreddits_at_request']]
subredDData.set_index('request_id', inplace=True)

# our training labels
trainLabel = allTData['requester_received_pizza']

devLabel = allDData['requester_received_pizza']

# What do these look like?
#print(list(flatData))
print(subredTData.shape)
#print(subredTData['requester_subreddits_at_request'][1])

# Create a corpus of subreddits to vectorize
trainCorpus = []
rTCorpus = []
rDCorpus = []

for index in range(len(subredTData)):
    trainCorpus.append(' '.join(subredTData['requester_subreddits_at_request'][index]))
    rTCorpus.append(' '.join(subredTData['requester_subreddits_at_request'][index]))

devCorpus = []
for index in range(len(subredDData)):
    devCorpus.append(' '.join(subredDData['requester_subreddits_at_request'][index]))
    rDCorpus.append(' '.join(subredDData['requester_subreddits_at_request'][index]))

# Baseline info
print("Percent of people who got pizza:", round(sum(trainLabel)/len(trainLabel),3))




In [ ]:
# Combine all text sources into a single corpus
fldTText = allTData[['request_id','request_text', 'request_text_edit_aware', 'request_title']]
fldDText = allDData[['request_id','request_text', 'request_text_edit_aware', 'request_title']]

trainCorpus = []
for index in range(len(subredTData)):
    a = ' '.join(subredTData['requester_subreddits_at_request'][index])
    b = (a, fldTText['request_text'][index], fldTText['request_text_edit_aware'][index],
        fldTText['request_title'][index])
    trainCorpus.append(' '.join(b))

devCorpus = []
for index in range(len(subredDData)):
    a = ' '.join(subredDData['requester_subreddits_at_request'][index])
    b = (a, fldDText['request_text'][index], fldDText['request_text_edit_aware'][index],
         fldDText['request_title'][index])
    devCorpus.append(' '.join(b))

# Print 3 examples  
print (trainCorpus[:1])
labels = trainLabel.astype(int)
labels = list(labels)
print(labels[:1])
print('-'*75)

print ('\n' , devCorpus[:1])
labels_dev = devLabel.astype(int)
labels_dev = list(labels_dev)
print(labels_dev[:1])

## Section 2: Simple Feature Selection and Pre-Processing

In [ ]:
# Simple Pre-Processing

def data_preprocessor(s):
    """
    Note: this function pre-processors data:
    (1) removes non-alpha characters
    (2) converts digits to 'number'
    (3) regularizes spaces (although CountVectorizer ignores this unless they are part of words)
    (4) reduces word size to n
    """

    s = [re.sub(r'[?|$|.|!|@|\n|(|)|<|>|_|-|,|\']',r' ',s) for s in s] # strip out non-alpha numeric char, replace with space
    s = [re.sub(r'\d+',r'number ',s) for s in s] # convert digits to number
    s = [re.sub(r' +',r' ',s) for s in s] # convert multiple spaces to single space
    
    # This sets word size to n=5
    num = 5
    def set_word(s):
        temp = []
        for s in s:
            x = s.split()
            z = [elem[:num] for elem in x]
            z = ' '.join(z)
            temp.append(z)       
        return temp
    
    s = set_word(s)
    
    return s

### Section 2.1 BiGrams 
After trying unigram and trigram vectorizers, the best results were found using bigrams in logistic regression


In [ ]:
# Try it with bigrams 
# Create the vectorizer
vectorizer = CountVectorizer(analyzer = 'word', ngram_range=(2, 2), 
                                token_pattern=r'\b\w+\b', min_df=1,stop_words = stop)
# Transform the corpus into vectorized trigrams
tVector = vectorizer.fit_transform(trainCorpus)
dVector = vectorizer.transform(data_preprocessor(devCorpus))
# How does it look?
print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", tVector.shape[1])
print ("First 5 feature Names:", vectorizer.get_feature_names()[1:6], "\n")
# Use the preprocessor and do the same
vectorizer_p = CountVectorizer(analyzer = 'word', ngram_range=(2, 2), 
                                  token_pattern=r'\b\w+\b', min_df=1,stop_words = stop)
tVector_p = vectorizer_p.fit_transform(data_preprocessor(trainCorpus))
dVector_p = vectorizer_p.transform(data_preprocessor(devCorpus))
# How does the pre-processed vector look?
print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", tVector_p.shape[1])
print ("First 5 feature Names:", vectorizer_p.get_feature_names()[1:6], "\n")

### Section 2.2 PCA

In [ ]:
# Try PCA

# k = 250 principal components    
n_comp = 250    
pca_mod = PCA(n_components = n_comp)
# PCA requires dense vectors:
tDense = tVector_p.todense()
# Get the principal components for the dense vector
pca_mod.fit(tDense)
# Find the fraction of the variance explained by each component
pcaVarRatio =  pca_mod.explained_variance_ratio_ 
pcaCumVarRatio =  np.cumsum(pca_mod.explained_variance_ratio_)
   
# Plot the fraction of variance explained by each component, and the cumulative percent 
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(range(len(pcaVarRatio)), pcaVarRatio, c = 'g', marker="s", label='Fraction')
ax1.scatter(range(len(pcaVarRatio)), pcaCumVarRatio, c = 'purple',marker="o", 
            label='Cumulative')
plt.legend(loc='upper left');
ax1.set_title('Fraction of Total Variance for k = 1 to 250');
plt.show()

### Section 2.3:  Status (Upvotes-Downvotes)

In [ ]:
## PARAMETER 1:  DIFFERENCE IN UPVOTES-DOWNVOTES (from time of request to time of retrieval) ##

# Training data

# Create separate database of desired columns
diffTrequest = allTData['requester_upvotes_minus_downvotes_at_request']
diffTretrieval = allTData['requester_upvotes_minus_downvotes_at_retrieval']

# Calculate upvote-downvote difference and add back into database
allTData['request_to_retrieval_upvotes_minus_downvotes'] = diffTretrieval - diffTrequest


# Development data (same process as above)
diffDrequest = allDData['requester_upvotes_minus_downvotes_at_request']
diffDretrieval = allDData['requester_upvotes_minus_downvotes_at_retrieval']
allDData['request_to_retrieval_upvotes_minus_downvotes'] = diffDretrieval - diffDrequest

In [ ]:
## CONVERT UPVOTES-DOWNVOTES SUBSETS INTO Z-SCORES FOR STANDARDIZATION ##

# Z-scores for training data

# Retrieve appropriate columns
statusTtrain = allTData[['requester_upvotes_minus_downvotes_at_request','requester_upvotes_minus_downvotes_at_retrieval',
                          'request_to_retrieval_upvotes_minus_downvotes']].copy()
statusTtrain.columns = ['Request', 'Retrieval', 'Request-Retrieval']
# Convert into Z-scores
for col in statusTtrain:
    zscore_col = col + " Z-score"
    statusTtrain[zscore_col] = (statusTtrain[col] - statusTtrain[col].mean() / statusTtrain[col].std(ddof=0))

# Z-scores for development data(same process as above)
statusDtrain = allDData[['requester_upvotes_minus_downvotes_at_request','requester_upvotes_minus_downvotes_at_retrieval',
                          'request_to_retrieval_upvotes_minus_downvotes']].copy()
statusDtrain.columns = ['Request', 'Retrieval', 'Request-Retrieval']

for col in statusDtrain:
    zscore_dev_col = col + " Z-score"
    statusDtrain[zscore_dev_col] = (statusDtrain[col] - statusDtrain[col].mean() / statusDtrain[col].std(ddof=0))

### Section 2.4:  Status (Upvotes-Downvotes)

In [ ]:
## PARAMETER 2:  TIMEDATE STAMP OF REDDIT POST ##


unix = allTData[['unix_timestamp_of_request_utc', 'requester_received_pizza']].copy()

# Convert from unix > datetime
unix['Datetime'] = pandas.to_datetime(unix['unix_timestamp_of_request_utc'], unit='s')
unix['Hour'] = unix['Datetime'].dt.hour
display(unix.head(5))

# set datetime as index
unix_index = unix.set_index('Datetime')
unix_index = unix_index.drop(['unix_timestamp_of_request_utc'], axis=1)

In [ ]:
# plot pizzas by month 
%matplotlib inline
unix_index_base = unix_index.drop(['Hour'], axis=1)
month_pizzas = unix_index_base.resample('M').sum()

month_pizzas_bar = month_pizzas.plot(kind='bar')

In [ ]:
# TIME ANALYSIS - HOUR OF REDDIT POST

# Removed first few months to minimize skew
hourly = unix_index['10/2012':'9/2013'].groupby('Hour')['requester_received_pizza'].sum()
print("Statistics of pizza success by hour\n")
print(hourly.describe())
print("\n\nPizza success by hour of the day")
hourly.plot('area')

### Section 2.5:  vaderSentiment Analysis

In [ ]:
import pandas as pd

titles = allTData['request_title']

    
df = pd.DataFrame(data = allTData)

df = df[['request_title', 'requester_received_pizza']]


titles = allTData['request_title']

def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    snt['compund']
    #print("{:-<40} {}".format(sentence, str(snt)))
    #print(snt['compound'])
    
scores = []

for title in titles:
    scores.append(analyser.polarity_scores(title)['compound'])

#print(scores[1:10])

df["Vader Scores"] = scores

#df.head(10)

df = df.drop('request_title', axis = 1)

print(df)

#df.head(10)

In [ ]:
import pandas as pd

titles = allDData['request_title']

df_d = pd.DataFrame(data = allDData)

df_d = df_d[['request_title', 'requester_received_pizza']]

def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    snt['compund']
    #print("{:-<40} {}".format(sentence, str(snt)))
    #print(snt['compound'])
    
scores_d = []

for title in titles:
    scores_d.append(analyser.polarity_scores(title)['compound'])

df_d["Vader Scores"] = scores_d

df_d = df_d.drop('request_title', axis = 1)

print(df_d)

In [ ]:
import pandas as pd

tTitles = allTData['request_title']
dTitles = allDData['request_title']

titleTSentiment = []
titleDSentiment = []

for title in tTitles:
    snt = analyser.polarity_scores(title)
    compoundScore = snt['compound']
    titleTSentiment.append(compoundScore)

titleTSentiment = pd.DataFrame(titleTSentiment)
    
for title in dTitles:
    snt = analyser.polarity_scores(title)
    compoundScore = snt['compound']
    titleDSentiment.append(compoundScore)

titleDSentiment = pd.DataFrame(titleDSentiment)

C = 100
modelLogit = LogisticRegression(penalty = 'l2', C = C)

trainLabel = allTData['requester_received_pizza']
devLabel = allDData['requester_received_pizza']

modelLogit.fit(titleTSentiment,trainLabel)
score_rep(devLabel,modelLogit.predict(titleDSentiment),'Logistic Regression, C = 0.001')


## Section 3: Models

### 3.1: Logistic Regression

In [ ]:
# Logistic Regression: bigrams
# Fit a logistic regression model to our bigrams, using l2 regularization and C = .5
c = .01
modelLogit = LogisticRegression(penalty='l2', C=c)
modelLogit.fit(tVector, trainLabel)

# Print out the classification report
print("Classification report raw bigrams:\n")
devPred = modelLogit.predict(dVector)
print(classification_report(devLabel, devPred))

# Print out the accuracy scode   
logitScore = round(modelLogit.score(dVector, devLabel), 4)
print("For C = ", c, "Logistic regression accuracy:", logitScore)

# Fit the pre-processed data
modelLogit_p = LogisticRegression(penalty='l2', C=c)
modelLogit_p.fit(tVector_p, trainLabel)

# Print out the classification report
print("\nClassification report processed bigrams:\n")
devPred = modelLogit_p.predict(dVector_p)
print(classification_report(devLabel, devPred))
    
# Print the accuracy score
logitScore = round(modelLogit_p.score(dVector_p, devLabel), 4)
print("For C = ", c, "Logistic regression accuracy:", logitScore)

### Section 3.2: Ensemble Model on PCA Features

In [ ]:
# Create an ensemble model on the PCA features
# Get the training features
pca_tran=pca_mod.transform(tDense)  
# Convert our dev features to be dense
dDense = dVector_p.todense()
# Get the dev features
pca_devTran=pca_mod.transform(dDense)  

# Set up lr_1
lr_1 = LogisticRegression(penalty='l2', C=0.01)
# Set up lr_2
lr_2 = LogisticRegression(penalty='l2', C=0.1)
# Set up lr_3
lr_3 = LogisticRegression(penalty='l2', C=0.5)

# Set up ensemble of the models
clf = EnsembleVoteClassifier(clfs=[lr_1, lr_2, lr_3], 
                             voting='soft', weights=[1,1, 1])

# Fit training data
clf.fit(pca_tran,trainLabel)
  
# Probabilities, predictions
devProb = clf.predict_proba(pca_devTran)
devPred = clf.predict(pca_devTran)

acc = sum(devPred == devLabel)/len(devLabel)
print("Logistic regression accuracy:", acc)
print(classification_report(devLabel, devPred))


### Section 3.3: Scoring Functions

In [ ]:
# Group scoring functions

def roc_curve1(y_true, y_pred_prob):
    """This function plots the ROC curve
    Inputs: y_true, correct label
            y_pred_prob, predicted probabilities
    """
    fpr, tpr, thr = roc_curve(y_true, y_pred_prob)
   
    plt.figure()
    plt.plot(fpr,tpr)
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.title("ROC Curve")
    plt.show()
    

def score_rep(y_true, y_pred, desc):
    """Function to print out comprehensive report for classification test
    Inputs: y_true, correct label
            y_pred, predicted label from model
            desc, description of model
    Output: classification report
    """
    print(desc)
    print("-"*75)
    print("Accuracy: ", metrics.accuracy_score(y_true, y_pred))
    print("Area under curve of ROC: ", metrics.roc_auc_score(y_true, y_pred))
    print("Classification report:\n")
    print(metrics.classification_report(y_true, y_pred))
    print("-"*75)

### Section 3.4: Status & Hour LR Model

In [ ]:
import warnings
warnings.filterwarnings('ignore') 

# FOR STATUS AND TIME BY HOUR
lr = LogisticRegression(penalty='l2', C=0.01)

train_labels = np.asarray(allTData['requester_received_pizza'])
dev_labels = np.asarray(allDData['requester_received_pizza'])

# setup training and development datasets
# train
stathourTData = pandas.concat([statusTtrain['Request-Retrieval Z-score'], allTData['unix_timestamp_of_request_utc']], axis=1, 
                              keys=['Request-Retrieval Z-score', 'unix_timestamp_of_request_utc'])
stathourTData['Datetime'] = pandas.to_datetime(stathourTData['unix_timestamp_of_request_utc'], unit='s')
stathourTData['Hour'] = stathourTData['Datetime'].dt.hour
stathourTData_ = stathourTData.drop(['Datetime', 'unix_timestamp_of_request_utc'], axis=1)
# display(stathourTData.head(5))

# development
stathourDData = pandas.concat([statusDtrain['Request-Retrieval Z-score'], allDData['unix_timestamp_of_request_utc']], axis=1, 
                              keys=['Request-Retrieval Z-score', 'unix_timestamp_of_request_utc'])
stathourDData['Datetime'] = pandas.to_datetime(stathourDData['unix_timestamp_of_request_utc'], unit='s')
stathourDData['Hour'] = stathourDData['Datetime'].dt.hour
stathourDData_ = stathourDData.drop(['Datetime', 'unix_timestamp_of_request_utc'], axis=1)
# display(stathourDData.head(5))

# fit LR models
train_stathour_model = lr.fit(stathourTData_, train_labels)
dev_stathour_labels = train_stathour_model.predict(stathourDData_)
stathour_score = train_stathour_model.score(stathourDData_, dev_labels)
stathour_F1score = metrics.f1_score(dev_stathour_labels, dev_labels)

print("\nScoring for Request-retrieval Z-scores and Hour")
print("Accuracy:", stathour_score)
print("F1 score:", stathour_F1score)
roc_curve1(dev_labels, train_stathour_model.predict_proba(stathourDData_)[:,0])
score_rep(dev_labels, dev_stathour_labels,"Logistic Regression, C = 0.01")

In [ ]:
# retrieve probabilities from text model where training label is True
text_model = modelLogit.fit(tVector_p, trainLabel)
prob = text_model.predict_proba(dVector_p)
prob_true = [float(y) for x,y in prob]
prob_true = pandas.DataFrame(prob_true)
prob_false = [float(x) for x,y in prob]
prob_false = pandas.DataFrame(prob_false)

# add as a parameter to status/hour dataframe
combinedTData = pandas.concat([stathourTData_, prob_true, prob_false], axis = 1)
combinedDData = stathourDData_

# fit LR models
train_comb_model = lr.fit(combinedTData, train_labels)
dev_comb_labels = train_train_comb_model.predict(combinedDData)
comb_score = train_comb_model.score(combinedDData, dev_labels)
comb_F1score = metrics.f1_score(dev_comb_labels, dev_labels)

print("\nScoring for Request-retrieval Z-scores and bucketed Hour:")
print("Accuracy:", comb_score)
print("F1 score:", comb_F1score)
roc_curve1(dev_labels, train_comb_model.predict_proba(combinedDData)[:,0])
score_rep(dev_labels, dev_comb_labels,"Logistic Regression, C = 0.01")
